<a href="https://colab.research.google.com/github/hendrikyong/CVNL_Assignment_1/blob/main/CVNL_P02_GP01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

# List files in the dataset folder
print("Path to dataset files:", path)
files = os.listdir(path)
print("Files in the dataset:", files)

100%|██████████| 1.03G/1.03G [00:17<00:00, 64.1MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1
Files in the dataset: ['asl_alphabet_train', 'asl_alphabet_test']


In [2]:
# Data preprocessing
import os
import numpy as np
import cv2

# Define the paths to the training and testing data directories
train_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_train/asl_alphabet_train'
test_dir = '/root/.cache/kagglehub/datasets/grassknoted/asl-alphabet/versions/1/asl_alphabet_test/asl_alphabet_test'

# List files in the train and test directories
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

print("Files in the training directory:", train_files)
print("Files in the testing directory:", test_files)

Files in the training directory: ['nothing', 'S', 'I', 'H', 'space', 'Q', 'J', 'X', 'G', 'B', 'D', 'R', 'N', 'P', 'A', 'K', 'M', 'E', 'O', 'del', 'W', 'C', 'L', 'F', 'T', 'V', 'Z', 'Y', 'U']
Files in the testing directory: ['nothing_test.jpg', 'W_test.jpg', 'I_test.jpg', 'B_test.jpg', 'L_test.jpg', 'space_test.jpg', 'O_test.jpg', 'S_test.jpg', 'V_test.jpg', 'N_test.jpg', 'H_test.jpg', 'R_test.jpg', 'J_test.jpg', 'T_test.jpg', 'Q_test.jpg', 'X_test.jpg', 'D_test.jpg', 'E_test.jpg', 'C_test.jpg', 'M_test.jpg', 'P_test.jpg', 'Z_test.jpg', 'G_test.jpg', 'F_test.jpg', 'Y_test.jpg', 'U_test.jpg', 'A_test.jpg', 'K_test.jpg']
